# Factor de friccion

In [1]:
# importando librerias 

import pandas as pd
import matplotlib 
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.random import check_random_state
from sklearn.metrics import mean_squared_error, r2_score
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import export_graphviz
import graphviz
import pandas as pd

In [2]:
# cargando datos

# importacion de DF
file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_25.txt'
df_ff_25 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_53.txt'
df_ff_53 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_74.txt'
df_ff_74 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_102.txt'
df_ff_102 = pd.read_csv(file_path, delimiter=',')

In [19]:
# definiendo conjuntos de train y test

# definiendo conjunto de train
df_ff_train = pd.concat([df_ff_25, df_ff_74], ignore_index=True)

# separando entre x e y
y_train = df_ff_train.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_train = df_ff_train.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','frictionFactor'])

# definiendo conjunto de test

y_test = df_ff_53.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_test = df_ff_53.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','frictionFactor'])

### Modelo original 

In [5]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=20, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    37.35      4.28094e+28       23         0.628484         0.678839     11.44m
   1    17.24      1.68328e+08       25         0.629673         0.665225      7.74m
   2    18.26       9.1841e+11       23         0.630923         0.653972     10.55m
   3    25.85      4.00302e+11       13         0.563529         0.547598      6.16m
   4    23.94      3.77601e+09       13         0.556433         0.541774      4.68m
   5    17.79           334315       13         0.555681          0.54854      6.03m
   6    13.63      8.31439e+07       13         0.551765         0.583786      3.69m
   7    11.34      2.01239e+07       13         0.550698         0.593388      3.12m
   8    10.83      1.00142e+08       13         0.549114         0.607643  

In [6]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 1.1110193928741372
R^2: 0.8043016746993664


In [ ]:
# comparar con modelo de Rafael

# 20*S^(-1.1)*Re^(-0.22)
def ffriction(S, Re):
    return 20*S**(-1.1)*Re*(-0.22)

#def ffriction_gp(S, Re):
#   return 

### Incorporar potencias

In [20]:
# S ^(-1.1) 
def pot1(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, np.power(x1, -1.1), 0)
    return result

pot1_fn = make_function(function=pot1, 
                        name='pot1', 
                        arity=1)

# 20* Re ^ (-0.22)
def pot2(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, 20*np.power(x1, -0.22), 0)
    return result

pot2_fn = make_function(function=pot2, 
                        name='pot2', 
                        arity=1)

In [21]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=25, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn, pot2_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    19.46       1.2489e+15        4          0.52396         0.514965     28.05m
   1    11.82      3.01331e+06        5         0.484317         0.457411     13.89m
   2     8.24      3.70547e+07        5          0.47922         0.503287     10.28m
   3     8.14          51184.1        5         0.476157         0.530848     10.59m
   4     5.74          53356.5       17          0.47276         0.460887      7.46m
   5     5.37      5.68701e+07        5         0.476578         0.527059      7.51m
   6     5.26      2.80269e+08        5         0.476737         0.525635     11.74m
   7     5.24          54549.3        5         0.476402         0.528651      9.59m
   8     5.24      1.21369e+08        5         0.474707         0.543898  

In [22]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 0.737637987361494
R^2: 0.8700702078373823


In [23]:
# comparar con modelo de Rafael

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re*(-0.22)

def fFriction_gp(S, Re):
   return S**(-1.1)*20*Re*(-0.22)  # misma expresion c:  

In [24]:
X_test['ff_pred'] = X_test.apply(lambda row: fFriction(row['K'], row['Rem']), axis=1)
X_test['ff_gp_pred'] = X_test.apply(lambda row: fFriction_gp(row['K'], row['Rem']), axis=1)

r2_ff = r2_score(y_test['frictionFactor'], X_test['ff_pred'])
mse_ff = mean_squared_error(y_test['frictionFactor'], X_test['ff_pred'])

# Calcular R^2 y MSE para cdrag_gp
r2_ff_gp = r2_score(y_test['frictionFactor'], X_test['ff_gp_pred'])
mse_ff_gp = mean_squared_error(y_test['frictionFactor'], X_test['ff_gp_pred'])

print(f"R^2 frictionFactor: {r2_ff}, MSE cdrag: {mse_ff}")
print(f"R^2 frictionFactor_gp: {r2_ff}, MSE cdrag_gp: {mse_ff_gp}")

R^2 frictionFactor: -79524813.05076136, MSE cdrag: 451478623.9962755
R^2 frictionFactor_gp: -79524813.05076136, MSE cdrag_gp: 451478623.9962755


Altos valores de error. Sin embargo, algo similar se menciona en la tesis de Rafael. 